In [1]:
for p in ("Knet", "Plots", "NBInclude")
    Pkg.installed(p) == nothing && Pkg.add(p);
end

In [2]:
using Knet, Plots, NBInclude;
nbinclude("deepppiutils.ipynb"); # loads trnper, devper, tstper, featuresDict, concatAB, train!, trainSep!, modelevaluation 
                                 # ygold, winit, minibatchi, predict, predictSep,loss,zeroone, report, lossgradient, params,
                                 # loadmodel

In [3]:
# the number of hidden units in the hidden layers of the DeepPPI-CON model
HIDDENS = Any[NOCONCAT, 512, 256, 128, 128, NOOUTPUTS]; 
NOEPOCH = 30;
BATCHSIZE = 64;
PDROP = (0, 0.2);

In [6]:
accuracy =[]
recall=[]
specifity=[]
precision= []
mcc=[]
f1=[]
npv=[]
accuracyt= recalli=specifityi=precisioni=mcci = 0.0
for i in 1:5
    #setseed(i);
    w = winit(HIDDENS...);
    
    #dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    dtrn, ddev = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    
    optims = params(w; optim="Momentum", lr=0.01, gamma=0.9);
    #@time trnloss, trnerr, tstloss, tsterr=trainSep!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    @time train!(w, optims, dtrn, predict, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    #trainSep!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    
    println("Dataset", i)
    println("Training: min. loss =",loss(w,dtrn,predict),", min. error =",zeroone(w,dtrn,predict))  
    println("Test: min. loss =",loss(w,ddev,predict),", min. error =",zeroone(w,ddev,predict))  
    
    accuracyt,recalli,specifityi,precisioni,mcci,f1i,npvi = modelevaluation(w, ddev, predict; p=true);
    push!(accuracy, accuracyt)
    push!(recall, recalli)
    push!(specifity, specifityi)
    push!(precision, precisioni)
    push!(mcc, mcci)
    push!(f1, f1i)
    push!(npv, npvi)
    
    writedlm("DeepPPI_ConModel"*string(i)*".csv", map(Array, w))
end

(:epoch, 10, :trn, 0.8327466475095785, :dev, 0.8095933787561146)
(:epoch, 20, :trn, 0.9761880055342699, :dev, 0.9027690426275332)
(:epoch, 30, :trn, 0.9845213388676032, :dev, 0.9048087002096435)
110.582605 seconds (70.33 M allocations: 2.425 GiB, 6.38% gc time)
Dataset1
Training: min. loss =0.0248191, min. error =0.008688537675606622
Test: min. loss =0.35022405, min. error =0.09310796645702313
TP: 3837 , TN: 3989 , FP: 450 , FN: 353
Model evaluation:
Accuracy : 0.9069417081933017
Precision : 0.8950314905528342
NPV : 0.9187010592080375
Sensitivity / Recall : 0.9157517899761336
Specifity : 0.8986258166253661
MCC : 0.8140550143941774
F1 : 0.9052730917889261
(:epoch, 10, :trn, 0.7378365793954875, :dev, 0.7256376659678546)
(:epoch, 20, :trn, 0.9764274691358025, :dev, 0.9009171907756814)
(:epoch, 30, :trn, 0.9929012345679012, :dev, 0.9100847309573724)
108.301782 seconds (69.81 M allocations: 2.390 GiB, 7.18% gc time)
Dataset2
Training: min. loss =0.019443903, min. error =0.006442901234567921

In [9]:
#summary(accuracy)
open("DeepPPI-Con_Scores.txt", "w") do f
    write(f, "Dataset \tAccuracy\t\t\tPrecision\t\t\tnpv      \t\t\tRecall   \t\t\tSpecifity\t\t\tMCC\n")
    write(f, "__________________________________________________________________________________________________________________________________\n")
    for i in 1:5
        write(f, "dataset"*string(i)*"\t"*string(accuracy[i]) *"\t"* string(precision[i]) *"\t"* string(npv[i]) *"\t"* string(recall[i]) *"\t"*  string(specifity[i]) *"\t"*  string(mcc[i]) *"\n")
    end
    write(f, "__________________________________________________________________________________________________________________________________\n")
    write(f, "Average"*"\t\t"*string(mean(accuracy)) *"\t"* string(mean(precision))  *"\t"* string(mean(npv)) *"\t"* string(mean(recall)) *"\t"*  string(mean(specifity)) *"\t"*  string(mean(mcc)) *"\n")
end;
#println("Accuracy", "   Precision", "   npv"," recall", "    specifity", "     mcc", "       f1")
#(hcat(accuracy, precision, npv, recall, specifity, mcc, f1))

In [11]:
w_sizes = Tuple{Int64,Int64}[(512, 1164) (512, 1) (256, 512) (256, 1) (128, 256) (128, 1) (128, 128) (128, 1) (2, 128) (2, 1)]
totalNoOfParams = 0;
for i in 1:length(w_sizes)
    totalNoOfParams += w_sizes[i][1] * w_sizes[i][2]
end
println("Total Number of trainable parameters: ", totalNoOfParams)

Total Number of trainable parameters: 777474


## The following code is used while training to test different hyper parameters

In [ ]:
plot([vcat(trnloss) vcat(tstloss)],ylim=(0.0,0.8), labels=[:trnMulti :trnloss_01 :trnloss_001 :trnloss_005 :tstMulti :tstloss_01 :tstloss_001 :tstloss_005],xlabel="Epochs",ylabel="Loss")  
#plot([vcat(trnloss)],ylim=(0.2,0.7),labels=[:trnMulti],xlabel="Epochs",ylabel="Loss")  

In [ ]:
plot([vcat(trnerr) vcat(tsterr)],ylim=(0.0,0.5), labels=[:trnMulti_1 :trnMulti_01 :trnMulti_001 :trnMulti_005 :tstMulti_1 :tstMulti_01 :tstMulti_001 :trnMulti_005],xlabel="Epochs",ylabel="Error")  
#plot([vcat(trnerr)],ylim=(0.05,0.45), labels=[:trnMulti],xlabel="Epochs",ylabel="Error")  

In [ ]:
# Training Set
trnacc = 0;
for (x, y) in dtrn
    ypred = predict(w,x)
    trnacc += accuracyi(ypred, y) 
end
print(trnacc/length(dtrn))

In [ ]:
# Test Set
tstacc = 0;
for (x, y) in dtst
    ypred = predict(w,x)
    tstacc += accuracyi(ypred, y) 
end
print(tstacc/length(dtst))

In [ ]:
# dev Set
devacc = 0;
for (x, y) in ddev
    ypred = predict(w,x)
    devacc += accuracyi(ypred, y) 
end
print(devacc/length(ddev))